<a href="https://colab.research.google.com/github/shibayu36/playground/blob/main/ai-reasoning-study/AI_Reasoning_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reasoningの学習


## セットアップ

左サイドバーのシークレットからOPENAI_API_KEYという名前でAPI Keyを入れておいてください

In [ ]:
!pip install transformers openai langchain langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [2]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] =  userdata.get('OPENAI_API_KEY')

## 実験1: チェーン・オブ・ソートで数学問題を解く

In [ ]:
from transformers import pipeline
generator = pipeline("text2text-generation", model="google/flan-t5-base", tokenizer="google/flan-t5-base")

Device set to use cpu


通常

In [ ]:
question = "Q: There are 23 apples in the cafeteria. We used 20 apples to make lunch and bought 6 more. How many apples are there in total?\nA: "
result = generator(question, max_length=100)
print(result[0]["generated_text"])

[{'generated_text': 'There are 23 + 20 = 36 apples in the cafeteria. There are 36 - 6 = 28 apples in total.'}]
There are 23 + 20 = 36 apples in the cafeteria. There are 36 - 6 = 28 apples in total.


CoTを使ったもの。あんま賢くなってない...

In [ ]:
cot_prompt = "Q: There are 23 apples in the cafeteria. We used 20 apples to make lunch and bought 6 more. How many apples are there in total?\nLet's think step by step.\nA: "
result = generator(cot_prompt, max_length=100)
print(result[0]["generated_text"])

There are 23 - 20 = 9 apples left. There are 9 + 6 = 19 apples left. The answer: 19.


gpt-3.5-turboを使う。そもそも賢すぎる。

In [3]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

response = client.responses.create(
    model="gpt-3.5-turbo",
    input="Q: ある店で製品を100ドルで販売しています。20%割引した後にさらに10%値上げしたら最終価格はいくらになるか？\nA: ",
)

print(response.output_text)

最初の価格から20%割引すると、100ドル × 0.2 = 20ドル割引されます。
したがって、割引後の価格は100ドル - 20ドル = 80ドルです。

次に、この価格にさらに10%値上げすると、80ドル × 0.1 = 8ドル値上がりします。
最終的な価格は80ドル + 8ドル = 88ドルになります。


## 実験2: LangChainを用いた簡単なReActエージェント

セットアップ

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

import langchain
langchain.debug = True

# OpenAIのLLM（GPT-4o-mini）を温度0で用意
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
# 電卓ツールをロード（LLMのインスタンスを渡す必要があります）
tools = load_tools(["llm-math"], llm=llm)
# ReAct (Zero-shot) エージェントを初期化
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


ReActで実行

In [ ]:
print("Human: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: ある店舗が製品を100ドルで販売しています。20%割引した後10%値上げしました。最終価格はいくら？\nThought:")

Human: Answer the following questions as best you can. You have access to the following tools:

Calculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: ある店舗が製品を100ドルで販売しています。20%割引した後10%値上げしました。最終価格はいくら？
Thought:


In [ ]:
question = "ある店舗が製品を100ドルで販売しています。20%割引した後10%値上げしました。最終価格はいくら？"
response = agent.run(question)
print("最終的なエージェントからの回答:", response)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "ある店舗が製品を100ドルで販売しています。20%割引した後10%値上げしました。最終価格はいくら？"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "ある店舗が製品を100ドルで販売しています。20%割引した後10%値上げしました。最終価格はいくら？",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThough

## 実験3: モデルの自己反省プロンプトを試す

1リットルの1/4を飲むと3/4が残り。3/4のうち1/3を飲むと2/4が残る。結果1/2リットルが答え。

しかしLLMはよく分からない計算をしてしまう。

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

question = "ジュースが1リットルありました。兄が全体の1/4を飲み、妹が残りの1/3を飲みました。残っているジュースは何リットルですか？"

response = client.responses.create(
    model="gpt-3.5-turbo",
    input=f"Q: {question}\nA: ",
    temperature=0.0
)
first_answer = response.output_text
print("初回回答: ", first_answer)

followup_prompt = f"先ほどあなたは「{question}」という質問に対して「{first_answer}」と答えましたが、その答えが正しいかよく考えてみてください。なぜその値段になるのか、手順を追って検証し、不整合があれば修正してください。\n最終的な答え: "
response = client.responses.create(
    model="gpt-3.5-turbo",
    input=followup_prompt,
    temperature=0.0
)
second_answer = response.output_text
print("2回目回答: ", second_answer)


初回回答:  ジュースが1リットルあったので、兄が1/4を飲んだので残りは3/4、それを妹が1/3を飲んだので残りは2/3です。したがって、残っているジュースは2/3リットルです。
2回目回答:  申し訳ありませんが、先ほどの答えに誤りがありました。正しい手順で計算し直します。

ジュースが1リットルあったとします。
兄が1/4を飲んだので、残りは1 - 1/4 = 3/4リットルです。
妹が残りの1/3を飲んだので、残りは3/4 × 2/3 = 6/12 = 1/2リットルです。

したがって、正しい答えは残っているジュースは1/2リットルです。


In [ ]:
followup_prompt = f"先ほどあなたは「{first_answer}」と答えましたが、その答えが正しいかよく考えてみてください。なぜその値段になるのか、手順を追って検証し、不整合があれば修正してください。\n最終的な答え: "

## 実験4: 推論APIの活用

In [4]:
from openai import OpenAI
import os

client = OpenAI()

prompt = """
'[1,2],[3,4],[5,6]' という形式の文字列として表された行列を受け取り、同じ形式で転置を出力する bash スクリプトを作成してください。
"""

response = client.responses.create(
    model="o4-mini",
    reasoning={"effort": "medium", "summary": "detailed"},
    input=[
        {
            "role": "user",
            "content": prompt
        }
    ],
)

print(response.output_text)

以下のような Bash スクリプト `transpose.sh` を作れば、入力文字列 `"[1,2],[3,4],[5,6]"` のような形式の行列を受け取り、同じ形式で転置を出力できます。

```bash
#!/usr/bin/env bash

# 使い方:
#   ./transpose.sh "[1,2],[3,4],[5,6]"
# 出力:
#   [1,3,5],[2,4,6]

if [ $# -lt 1 ]; then
  echo "Usage: $0 \"[a,b,...],[c,d,...],...\"" >&2
  exit 1
fi

input="$1"

# 1) "[...],[...],..." → 行ごと (改行区切り) の "x,y,z,..." へ
#    - 先頭の '[' と末尾の ']' を削る
#    - "],[" を改行に置換
mapfile -t rows < <(
  echo "$input" \
    | sed 's/^\[//; s/\]$//; s/\],\[/\'$'\n/g'
)

# 行数・列数を取得（要素数が揃っている前提）
R=${#rows[@]}
C=$(awk -F, 'NR==1{print NF}' <<< "${rows[0]}")

# 2) transpose を計算
#    trans[i][j] = rows[j][i]
declare -a out_rows
for (( i = 0; i < C; i++ )); do
  row="["

  for (( j = 0; j < R; j++ )); do
    # 行 j を comma で分割して配列 elems に
    IFS=, read -ra elems <<< "${rows[j]}"
    row+=${elems[i]}
    if [ $j -lt $((R-1)) ]; then
      row+=,
    fi
  done

  row+="]"
  out_rows+=("$row")
done

# 3) 出力形式に戻して表示
( IFS=,; echo "${out_rows[*]}" )
```

保存したら実行権限を付与してお使いください:

```ba

In [5]:
import json

# Pydantic モデルを辞書に変換し、JSON として整形出力
print(json.dumps(response.model_dump(), indent=2))

{
  "id": "resp_681a8f13ccb081929b4d7eac14a1dd4a08cbb10742db1562",
  "created_at": 1746571027.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "o4-mini-2025-04-16",
  "object": "response",
  "output": [
    {
      "id": "rs_681a8f146aa48192b293835d73a69f4608cbb10742db1562",
      "summary": [
        {
          "text": "**Creating a bash script for transposing matrices**\n\nThe user wants a bash script that can take a string representation of a matrix like \"[1,2],[3,4],[5,6]\" and return its transpose in the same format\u2014so output should be \"[1,3,5],[2,4,6].\" It seems like they\u2019re mostly dealing with a 2D matrix. I need to parse the input string, maybe using tools like sed or awk. I can read the input, convert the format, and then reconstruct it to get the desired output.",
          "type": "summary_text"
        },
        {
          "text": "**Drafting a bash script for matrix transposition**\n\nI need to create a 